### Actividad 2
Realizar un escaneo del tipo SYN Stealth al servidor `innova.uv.cl`, interprete y 
muestre los resultados. Identifique el sistema operativo.|

In [5]:
import subprocess
import os
from datetime import datetime

# Definir las variables para el comando nmap
scan_types = ["-sS"]  # Realiza un escaneo SYN Stealth
timing_template = "-T4"  # Establece la velocidad del escaneo a "agresiva"
advanced_options = "-A"  # Habilita la detección de OS, versión de servicio y script
verbosity_level = "-v"  # Incrementa la verbosidad del output
target = "innova.uv.cl"  # Especifica el host objetivo del escaneo


# Crear la carpeta para almacenar los resultados si no existe
folder_name = "temp"
os.makedirs(folder_name, exist_ok=True)

# Función para formatear el nombre del archivo basado en el tipo de escaneo y la fecha actual
def get_output_filename(scan_type):
    date_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    return os.path.join(folder_name, f"nmap_output_{scan_type.strip('-')}_{date_str}.xml")

# Ejecutar los escaneos y guardar cada resultado en un archivo separado
for scan_type in scan_types:
    output_file = get_output_filename(scan_type)
    output_format = f"-oX {output_file}"
    
    # Formatear el comando usando f-strings
    command = f"nmap {scan_type} {target} {timing_template} {advanced_options} {verbosity_level} {output_format}"

    # Ejecutar el comando nmap
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    # Imprimir la salida y los errores
    print(f"Comando: {command}")
    print(result.stdout)
    if result.stderr:
        print("Error:", result.stderr)

Comando: nmap -sS innova.uv.cl -T4 -A -v  -oX temp\nmap_output_sS_2024-05-11_13-00-46.xml
Starting Nmap 7.95 ( https://nmap.org ) at 2024-05-11 13:00 Hora est. Sudamérica Pacífico
NSE: Loaded 157 scripts for scanning.
NSE: Script Pre-scanning.
Initiating NSE at 13:00
Completed NSE at 13:00, 0.00s elapsed
Initiating NSE at 13:00
Completed NSE at 13:00, 0.00s elapsed
Initiating NSE at 13:00
Completed NSE at 13:00, 0.00s elapsed
Initiating Ping Scan at 13:00
Scanning innova.uv.cl (200.14.249.16) [4 ports]
Completed Ping Scan at 13:00, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 13:00
Completed Parallel DNS resolution of 1 host. at 13:00, 0.02s elapsed
Initiating SYN Stealth Scan at 13:00
Scanning innova.uv.cl (200.14.249.16) [1000 ports]
Discovered open port 80/tcp on 200.14.249.16
Discovered open port 443/tcp on 200.14.249.16
Discovered open port 22/tcp on 200.14.249.16
Discovered open port 3306/tcp on 200.14.249.16
Completed SYN Stealth Scan at 13:00, 

Desde la perspectiva de la ciberseguridad y seguridad de sistemas, el escaneo realizado con `nmap` al servidor `innova.uv.cl` revela información crucial para evaluar y fortalecer la postura de seguridad del servidor. A continuación, te proporciono una conclusión simple y breve basada en los resultados del escaneo:

### Conclusión de Ciberseguridad:

1. **Puertos y Servicios Activos**:
   - **SSH (22/tcp)**: Está abierto y usa OpenSSH 7.8. Es vital asegurarse de que solo se permitan conexiones desde direcciones IP confiables y que se utilicen técnicas de autenticación robustas como claves SSH.
   - **HTTP (80/tcp) y HTTPS (443/tcp)**: Ambos puertos están abiertos, indicando que el servidor está sirviendo contenido web. Es importante configurar correctamente el servidor web Apache para mitigar vulnerabilidades y asegurarse de que todo el tráfico HTTP sea redirigido a HTTPS para cifrar las comunicaciones.
   - **MySQL (3306/tcp)**: Este puerto está abierto pero marcado como "unauthorized", lo cual es positivo ya que significa que no se permitió el acceso sin autenticación durante el escaneo. Sin embargo, debe asegurarse de que la base de datos esté protegida adecuadamente contra ataques de inyección SQL y acceso no autorizado.

2. **Detección del Sistema Operativo**:
   - Los resultados sugieren que el servidor podría estar ejecutando FreeBSD o macOS. Es crucial mantener el sistema operativo actualizado con los últimos parches de seguridad para protegerse contra vulnerabilidades conocidas.

3. **Configuraciones y Certificados SSL**:
   - El certificado SSL está activo bajo el nombre común `*.uv.cl`, lo que es adecuado para proteger múltiples subdominios bajo `uv.cl`. Es importante revisar la validez y configuración del certificado regularmente para prevenir problemas de seguridad como ataques de intermediario.

4. **Medidas de Seguridad Adicionales**:
   - Implementar un WAF (Web Application Firewall) para proteger las aplicaciones web.
   - Utilizar herramientas de monitoreo y alerta para detectar y responder a actividades sospechosas o maliciosas en tiempo real.
   - Asegurar que todos los servicios innecesarios estén deshabilitados para minimizar la superficie de ataque.

5. **Riesgos Potenciales**:
   - La exposición de múltiples servicios incrementa el riesgo de ataques. Cada puerto abierto debe ser justificado y asegurado.
   - La implementación de medidas de seguridad debe ser revisada y probada regularmente para asegurar que sigan siendo efectivas frente a las amenazas emergentes.